## img url 받아오는 다양한 방법 1 

이미지 데이터를 받아오려는 페이지의 이미지가 링크로 연결된 배경이미지인 경우

(소스코드에서 a 태그안에 style의 background url로 지정되어있는경우)


In [ ]:
#step1. 필요한 모듈과 라ㅣ브러리를 로딩하고 검색어를 입력받는다
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import urllib

import math
import time
import sys
import os

f_dir = input('사진을 저장 할 폴더를 지정하세요:(C:\\Study\\Python\\notebook\\data\\)')
query_txt = '사진저장'

In [ ]:
#step2. 파일을 저장 할 폴더를 생성한다
now = time.localtime()
s = '{}-{}-{}-{}-{}' .format(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min)

os.makedirs(f_dir+s+'-'+query_txt)
f_dir_name = f_dir+s+'-'+query_txt
print(f_dir_name)

In [ ]:
#step3. 크롬 드라이버를 사용해서 웹 브라우저를 실행한다
path = 'C:\\Study\\Python\\datadown\\chromedriver.exe'
driver = webdriver.Chrome(path)

s_time = time.time() # 크롤링 시작 시간을 위한 타임 스탬프를 찍기
driver.get("https://korean.visitkorea.or.kr/detail/cs_detail_cos.do?cotid=1923378b-3e45-4ae9-8610-c8235636ba5e&big_category=A01&mid_category=A0100&big_area=39")
time.sleep(2)

#pc version으로 진행위해 window크기 maximize
#내용을 불러오기위해 특정 경로 click시, xpath가 pc vr.과 mobile vr.이 다르다.
driver.maximize_window() 

In [ ]:
#step4. 스크롤다움 함수를 생성한 후 실행하기
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #0부터 페이지끝까지
    time.sleep(1) #내려갈 시간주기
    
scroll_down(driver)

In [ ]:
# 같은 <a href="#cosTab01">의 xpath를 찾아야하지만 pc version, mobile version이 따로있음:
# pc version은 <div class="pc">안에, mobile은 동일 레벨 다음 div로 <div class="mo js_slider">안에 
# 각각 version의 내용이 담겨있다

# full hierarchy:
# <div id = "contents">
# 	<div class = "course_detail">
# 		<div class = "relation_cos">
# 			<div class="cos_wrap">
# 				<div class = "pc">
# 				<div class = "mo js_slider">

# step5에서 이미지 추출할때에 클릭하는 a태그의 xpath 경로가 다음과 같다:

# xpath of <a href="#cosTab01"> for pc:
# //*[@id="contents"]/div[2]/div[6]/div[1]/div[1]/div/ul/li[1]/a

# xpath of <a href="#cosTab01"> for mobile:
# //*[@id="contents"]/div[2]/div[6]/div[1]/div[2]/div/ul/li[1]/a

In [ ]:
#Step 5. 이미지 추출하여 저장하기
file_no=0 #이미지 순번
img_src2=[] #src값들 넣어둘 리스트

num_thumbs=4
for x in range(1,num_thumbs+1):
    driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[6]/div[1]/div[1]/div/ul/li[%s]/a'%x).click()
    
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    tag = soup.find('div',"cos_cont active")
    tag2 = tag.find('div','info_area pc').find_all('a')

    for i in tag2:
        img_src = i['style'].split("(")[1].split(")")[0]
        img_src2.append(img_src) #리스트에 저장

#이미지를 저장하려는 위치로 디렉토리를 변경
os.chdir(f_dir+s+'-'+query_txt)

#img_src2리스트 길이만큼 urlretreieve를 반복
for i in range(0,len(img_src2)):
    try:
        urllib.request.urlretrieve(img_src2[i],str(file_no)+'.jpg')
    except:
        continue
    file_no+=1
    time.sleep(0.5)
    print("%s번째 이미지 저장중입니다."%file_no)
        
img_src2

In [ ]:
#step6. 요약정보 출력하기
e_time = time.time()
d_time = e_time - s_time

print("=" *70)
print("총 소요시간은 %s 초 입니다 " %round(d_time,1))
print("총 저장 건수는 %s 건 입니다 " %file_no)
print("파일이 저장된 폴더 경로는 %s 입니다" %f_dir_name)
print("=" *70)
 
driver.close( )